# Recommendation System

данные о фильмах [tmdb_5000_movies](https://files.sberdisk.ru/s/te4QbzdxKgsFQXA) и каст фильмов
[tmdb_5000_credits](https://files.sberdisk.ru/s/H9oRuXQt5mFz3T9). 

In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate
from typing_extensions import dataclass_transform

In [2]:
df_movies = pd.read_csv('../datasets/tmdb_5000_movies.csv')
df_credits = pd.read_csv('../datasets/tmdb_5000_credits.csv')

released_df = pd.concat([df_movies, df_credits], axis=1)
released_df = released_df[released_df['status'] == 'Released']

print('Количество фильмов, оставшихся после фильтрации:', released_df.shape[0])

Количество фильмов, оставшихся после фильтрации: 4795


рекомендации фильмов с лучшими оценками пользователей 

In [3]:
def weighted_rating(df, C, m):
    v = df['vote_count']
    R = df['vote_average']
    return (v / (v + m) * R) + (m /(v + m) * C)

m = released_df['vote_count'].quantile(0.95)
C = released_df['vote_average'].mean()

released_df['simple_score'] = released_df.apply(lambda x: weighted_rating(x, C, m), axis=1)
#released_df['simple_score']

IMDB_top5 = list(released_df.sort_values(by=['simple_score'], ascending=False)[:5]['original_title'].values)
print('Топ-5 фильмов по версии IMDB:', *IMDB_top5, sep='\n')

Топ-5 фильмов по версии IMDB:
The Shawshank Redemption
The Dark Knight
Fight Club
Inception
Pulp Fiction


**Content Based Filtering** (Фильтрация на основе содержания) 

In [4]:
released_df['overview'] = released_df['overview'].fillna(value='')

vectorizer = TfidfVectorizer(stop_words='english')
Tf_Idf = vectorizer.fit_transform(released_df['overview'])

print('Размер матрицы Tf-Idf:', Tf_Idf.shape)

Размер матрицы Tf-Idf: (4795, 20970)


использовали векторизатор TF-IDF на предыдущем шаге, достаточно вычислить скалярное произведение, которое и даст оценку косинусного сходства

In [5]:
cosine_sim = linear_kernel(Tf_Idf, Tf_Idf)
print('Размер матрицы cosine_sim:', cosine_sim.shape)

Размер матрицы cosine_sim: (4795, 4795)


In [6]:
def get_recommendations(movies_dataset, title, cosine_sim, top_k=10):
    indices = pd.Series(movies_dataset.index, index = movies_dataset['original_title'])
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top_k + 1]
    movie_indices = [i[0] for i in sim_scores]
    return list(movies_dataset['original_title'].iloc[movie_indices])


print('Топ-5 фильмов, описание которых похоже на описание фильма "Saving Private Ryan":',
      *get_recommendations(released_df, title='Saving Private Ryan', cosine_sim=cosine_sim, top_k=5), sep='\n')

Топ-5 фильмов, описание которых похоже на описание фильма "Saving Private Ryan":
The Great Raid
The Monuments Men
The Expendables 2
Abandoned
The Train


**Collaborative Filtering** (Коллаборативная фильтрация)

In [7]:
rating_data = pd.read_csv('../datasets/ratings.csv', sep=',')
min_rating = rating_data['rating'].min()
max_rating = rating_data['rating'].max()

reader = Reader(rating_scale=(min_rating, max_rating))
data = Dataset.load_from_df(rating_data[['userId', 'movieId', 'rating']], reader)
results = cross_validate(algo=SVD(), data=data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
#results

print('')
print('Среднее значение RMSE:', round(results['test_rmse'].mean(), 3))
print('Среднее значение MAE:', round(results['test_mae'].mean(), 3))

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8951  0.9034  0.8920  0.8952  0.8982  0.8968  0.0038  
MAE (testset)     0.6874  0.6944  0.6878  0.6879  0.6928  0.6901  0.0029  
Fit time          1.00    1.03    1.02    1.01    1.04    1.02    0.02    
Test time         0.17    0.10    0.12    0.09    0.11    0.12    0.03    

Среднее значение RMSE: 0.8968164117781077
Среднее значение MAE: 0.6900521490266005
